In [131]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time, json, os, csv

from scipy.stats import linregress
from citipy import citipy
from pprint import pprint

# Hi Markers, please use your onw API keys to run the script, thank you!
from api_keys import weather_api_key
from api_keys import g_key

In [132]:
# Define function to set data type
def set_integer(data_type, pddf):
    for row in data_type:
        pddf[row] = pddf[row].astype(int)

# integer = int

In [133]:
# Read data files
house_data_path = "data_source/Melbourne_housing_FULL.csv"
school_data_path = "data_source/dv309_schoollocations2021.csv"
crime_rate_path = "data_source/Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2021.xlsx"

house_data = pd.read_csv(house_data_path)
school_data = pd.read_csv(school_data_path, encoding = "cp1252")
crime_data = df = pd.read_excel (crime_rate_path, sheet_name = "Table 03")

# Set output file path
house_file = "data_source/cleaned_source_data/house_cleaned.csv"
school_file = "data_source/cleaned_source_data/school_cleaned.csv"
crime_file = "data_source/cleaned_source_data/crime_cleaned.csv"

In [134]:
house_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [135]:
# cleaning house data
house_data_cleaned = house_data.dropna(subset = ["Price"])
house_data_cleaned = house_data_cleaned.drop(["Bedroom2"], axis = "columns")
house_data_cleaned = house_data_cleaned.fillna(0)

# Set up a list for data type
column_list = ["Postcode",
               "Bathroom",
               "Car",
               "Landsize",
               "BuildingArea",
               "YearBuilt",
               "Propertycount",
               "Rooms"]

set_integer(column_list, house_data_cleaned)

house_data_cleaned["Price"] = house_data_cleaned.apply(lambda x: "{:,.0f}".format(x["Price"]), axis = 1)

In [136]:
house_data_cleaned.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,1,1,202,0,0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,1,0,156,79,1900,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019
4,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,4/03/2017,2.5,3067,2,0,134,150,1900,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019
5,Abbotsford,40 Federation La,3,h,"850,000",PI,Biggin,4/03/2017,2.5,3067,2,1,94,0,0,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019
6,Abbotsford,55a Park St,4,h,"1,600,000",VB,Nelson,4/06/2016,2.5,3067,1,2,120,142,2014,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019


In [137]:
# house_data_cleaned['Postcode'] = house_data_cleaned['Postcode'].astype(int)
# house_data_cleaned['Bathroom'] = house_data_cleaned['Bathroom'].astype(int)
# house_data_cleaned['Car'] = house_data_cleaned['Car'].astype(int)
# house_data_cleaned['Landsize'] = house_data_cleaned['Landsize'].astype(int)
# house_data_cleaned['BuildingArea'] = house_data_cleaned['BuildingArea'].astype(int)
# house_data_cleaned['YearBuilt'] = house_data_cleaned['YearBuilt'].astype(int)
# house_data_cleaned['Propertycount'] = house_data_cleaned['Propertycount'].astype(int)

# house_data_cleaned.head()

In [138]:
house_data_cleaned.to_csv(house_file, index = False, encoding = "utf-8")

In [139]:
school_data_cleaned = school_data.drop(["SCHOOL_NO",
                                        "Address_Line_2",
                                        "Postal_Address_Line_1",
                                        "Postal_Address_Line_2",
                                        "Postal_Town",
                                        "Postal_State",
                                        "Postal_Postcode"], axis = "columns")

school_data_cleaned = school_data_cleaned.rename({"X": "Lng",
                                                  "Y": "Lat",
                                                  "Address_Line_1": "Address",
                                                  "Address_Town": "Suburb",
                                                  "Address_State": "State",
                                                  "Address_Postcode": "Postcode"}, axis = "columns")


school_data_cleaned.head()

,Education_Sector,Entity_Type,School_Name,School_Type,School_Status,Address,Suburb,State,Postcode,Full_Phone_No,LGA_ID,LGA_Name,Lng,Lat
0,Government,1,Alberton Primary School,Primary,O,21 Thomson Street,Alberton,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,Allansford and District Primary School,Primary,O,Frank Street,Allansford,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,Avoca Primary School,Primary,O,118 Barnett Street,Avoca,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,Avenel Primary School,Primary,O,40 Anderson Street,Avenel,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,Warrandyte Primary School,Primary,O,5-11 Forbes Street,Warrandyte,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [140]:
school_data_cleaned.to_csv(school_file, index = False, encoding = "utf-8")

In [141]:
crime_data

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2021,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,1
1,2021,March,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
2,2021,March,Alpine,3691,Gundowring,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1
3,2021,March,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1
4,2021,March,Alpine,3691,Kergunyah South,B Property and deception offences,B40 Theft,B49 Other theft,1
...,...,...,...,...,...,...,...,...,...
319368,2012,March,Yarriambiack,3487,Lascelles,B Property and deception offences,B50 Deception,B53 Obtain benefit by deception,1
319369,2012,March,Yarriambiack,3488,Speed,B Property and deception offences,B40 Theft,B49 Other theft,2
319370,2012,March,Yarriambiack,3489,Tempy,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1
319371,2012,March,Yarriambiack,3489,Tempy,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,1


In [142]:
crime_data_cleaned = crime_data.loc[(crime_data["Year"] >= 2016) &
                                    (crime_data["Year"] <= 2018)].dropna()

crime_data_cleaned.head()

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
102311,2018,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,1
102312,2018,March,Alpine,3691,Dederang,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,1
102313,2018,March,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,1
102314,2018,March,Alpine,3691,Dederang,C Drug offences,C10 Drug dealing and trafficking,C12 Drug trafficking,1
102315,2018,March,Alpine,3691,Glen Creek,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,2


In [143]:
crime_data_cleaned.to_csv(crime_file, index = False, encoding = "utf-8")